### 传感器和数据

#### 1. 设置传感器

In [8]:
import carla
from numpy import random

client = carla.Client('localhost', 2000)
world = client.load_world('Town07_Opt')
# 找到传感器的蓝图。
blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
# 修改蓝图属性以设置图像分辨率和视野。
blueprint.set_attribute('image_size_x', '1920')
blueprint.set_attribute('image_size_y', '1080')
blueprint.set_attribute('fov', '110')
# 设置传感器两次采集数据之间的时间间隔（以秒为单位）。
blueprint.set_attribute('sensor_tick', '1.0')

#### 2. 传感器和车的连接方式
    刚性连接（Rigid attachment）。运动严格受限于其父级位置。这是从仿真中检索数据的合适连接方式。
    弹簧臂连接（SpringArm attachment）。运动较为平缓，加减速幅度较小。建议仅使用此连接方式录制仿真视频。运动流畅，在更新摄像机位置时避免出现“跳动”。
    弹簧臂幽灵连接（SpringArmGhost attachment）。与前者类似，但不进行碰撞检测，因此摄像机或传感器可以穿过墙壁或其他几何体。

In [9]:
vehicle_blueprint = world.get_blueprint_library().find('vehicle.audi.tt')
spawn_point = random.choice(world.get_map().get_spawn_points())
# 现在我们来生成车辆。
world.spawn_actor(vehicle_blueprint, spawn_point)
vehicle = world.try_spawn_actor(vehicle_blueprint, spawn_point)

# 绑定传感器
transform = carla.Transform(carla.Location(x=0.8, z=1.7))
sensor = world.spawn_actor(blueprint, transform, attach_to=vehicle,attachment_type=carla.libcarla.AttachmentType.Rigid)

#### 3. 监听
    每个传感器都有一个 listen() 方法。每次传感器获取数据时，都会调用此方法。
    回调函数是一个 lambda 函数，用于描述传感器在获取数据后应该执行的操作。该函数必须以获取到的数据作为参数。
    `is_listening()` 是一个传感器方法，用于检查传感器是否已注册监听回调。
    `stop()` 是一个传感器方法，用于停止传感器监听。
    `sensor_tick` 是一个蓝图属性，用于设置接收数据之间的仿真时间间隔。

In [ ]:
# 每次相机生成新图像时，都会调用 do_something() 函数。
sensor.listen(lambda data: do_something(data))
...
# 该碰撞传感器会在每次检测到碰撞时打印数据。
def callback(event):
    for actor_id in event:
        vehicle = world_ref().get_actor(actor_id)
        print('Vehicle too close: %s' % vehicle.type_id)
sensor02.listen(callback)

#### 4. 数据
    传感器数据属性 	类型 	            描述
    frame 	        int 	            测量发生时的帧号。
    timestamp       double 	            自该集开始以来，以模拟秒为单位记录测量时间戳。
    transform       carla.Transform     测量时传感器的世界参考坐标。

#### 5. 传感器类型

##### 5.1 照相机Cameras
    传感器 	                输出 	                概述
    Depth 	                carla.Image 	        以灰度图的形式渲染视野中元素的深度。
    RGB 	                carla.Image 	        提供清晰的周围环境视野，看起来就像一张普通的场景照片。
    Optical Flow 	        carla.Image 	        渲染相机中每个像素的运动。
    Semantic segmentation   carla.Image 	        根据标签，以特定颜色渲染视野中的元素。
    Instance segmentation   carla.Image 	        根据元素的标签和唯一的对象 ID，以特定颜色渲染视野中的元素。
    DVS 	                carla.DVSEventArray 	以事件流的形式异步测量亮度强度的变化。

##### 5.2 探测器Detectors
    传感器 	                        输出 	                        概述
    Collision（碰撞传感器） 	        carla.CollisionEvent 	        获取其父角色与其他角色之间的碰撞信息。
    Lane invasion（车道入侵传感器） 	carla.LaneInvasionEvent         当其父节点越过车道线时进行注册。
    Obstacle（障碍物传感器） 	        carla.ObstacleDetectionEvent 	探测其父级前方可能存在的障碍物。

##### 5.3其它
    传感器 	        输出 	                                    概述
    GNSS 	        carla.GNSSMeasurement 	                    获取传感器的地理位置。
    IMU 	        carla.IMUMeasurement 	                    由加速度计、陀螺仪和指南针组成。
    LIDAR 	        carla.LidarMeasurement 	                    旋转式激光雷达。生成包含每个点的坐标和强度信息的4D点云，用于模拟周围环境。
    Radar 	        carla.RadarMeasurement 	                    二维点云建模视野中的元素及其相对于传感器的运动。
    RSS 	        carla.RssResponse                           根据安全检查结果，修改应用于车辆的控制器。此传感器的运行方式与其他传感器不同，并有专门的 RSS 文档对其进行说明。
    Semantic LIDAR 	carla.SemanticLidarMeasurement 	            旋转式激光雷达。生成包含实例分割和语义分割等额外信息的三维点云。